In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc 
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
def make_datetime(x):
    x=str(x)
    year=int(x[:4])
    month=int(x[4:6])
    day=int(x[6:8])
    hour=int(x[8:10])
    minute=int(x[10:12])
    sec=int(x[12:])
    return dt.datetime(year, month, day, hour)

In [3]:
def string2num(x):
    x=re.sub(r"[^0-9]+", '', str(x))
    if x=='':
        return 0
    else:
        return int(x)

In [4]:
train_err=pd.read_csv('train_err_data.csv')
train_err

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0
...,...,...,...,...,...,...
16554658,24999,20201130163051,model_3,05.15.2138,15,1
16554659,24999,20201130172625,model_3,05.15.2138,16,1
16554660,24999,20201130172625,model_3,05.15.2138,4,0
16554661,24999,20201130172631,model_3,05.15.2138,4,0


In [5]:
train_err.loc[3825744]

user_id              13639
time        20201121191718
model_nm           model_2
fwver           04.33.1261
errtype                  5
errcode                NaN
Name: 3825744, dtype: object

In [6]:
train_err['errcode_New'] = np.where(pd.notnull(train_err['errcode']) == True, train_err['errcode'], 'B-A8002')

In [7]:
train_err.loc[3825744]

user_id                 13639
time           20201121191718
model_nm              model_2
fwver              04.33.1261
errtype                     5
errcode                   NaN
errcode_New           B-A8002
Name: 3825744, dtype: object

In [8]:
train_err['errtype'].value_counts()

23    2276515
31    1985294
22    1865361
15    1604951
4     1296750
16    1249376
32    1130564
5      952920
40     730378
28     544306
27     540804
26     536401
12     320181
11     307030
14     252015
33     178477
34     165536
10     133403
41     104138
7       51993
6       50482
17      46009
24      39071
42      34374
3       30758
2       26106
13      22843
1       21545
39      15820
35      10187
37       9336
36       9317
38       4539
18       2671
20       2080
25       1184
19        664
21        607
30        402
8         166
9         109
Name: errtype, dtype: int64

In [9]:
t=train_err[train_err['user_id']==13639]
t

,user_id,time,model_nm,fwver,errtype,errcode,errcode_New
3823076,13639,20201101000004,model_2,04.33.1185,2,0,0
3823077,13639,20201101000533,model_2,04.33.1185,2,0,0
3823078,13639,20201101002914,model_2,04.33.1185,2,1,1
3823079,13639,20201101002924,model_2,04.33.1185,2,0,0
3823080,13639,20201101003121,model_2,04.33.1185,2,1,1
...,...,...,...,...,...,...,...
3825960,13639,20201130220302,model_2,04.33.1261,16,1,1
3825961,13639,20201130220303,model_2,04.33.1261,40,0,0
3825962,13639,20201130220306,model_2,04.33.1261,31,1,1
3825963,13639,20201130220318,model_2,04.33.1261,31,0,0


In [10]:
tt=t[t['model_nm']=='model_2']

In [11]:
ttt=tt[tt['fwver']=='04.33.1261']

In [12]:
ttt=ttt[ttt['errtype']==5]

In [13]:
ttt['errcode'].value_counts()

B-A8002    61
H-51042     2
40013       1
Name: errcode, dtype: int64

In [14]:
train_user_number=len(train_err['user_id'].unique())
train_user_id_min = 10000
train_user_number

15000

In [15]:
len(np.sort(train_err['errtype'].unique()))

41

In [16]:
id_error=train_err[['user_id','errtype']].values
id_error

array([[10000,    15],
       [10000,    12],
       [10000,    11],
       ...,
       [24999,     4],
       [24999,     4],
       [24999,    15]], dtype=int64)

In [17]:
error=np.zeros((train_user_number, 42))
error

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [18]:
for person_idx, err in tqdm(id_error):
    error[person_idx-train_user_id_min, err-1]+=1
error

100%|███████████████████████████████████████████████████████████████████| 16554663/16554663 [03:10<00:00, 86929.33it/s]


array([[  0.,   0.,   8., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ..., 113.,  56.,   1.],
       [  0.,   0.,   2., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,  58.,   8.,   5.],
       [  0.,   0.,   0., ...,   6.,   0.,   0.],
       [  0.,   0.,   4., ...,   0.,   0.,   0.]])

In [19]:
train_prob=pd.read_csv('train_problem_data.csv')
train_prob

,user_id,time
0,19224,20201102200000
1,23664,20201116140000
2,15166,20201114130000
3,12590,20201108210000
4,15932,20201103210000
...,...,...
5424,20167,20201125120000
5425,16270,20201110120000
5426,19114,20201106230000
5427,21505,20201104110000


In [20]:
problem=np.zeros(15000)
problem[train_prob['user_id'].unique()-10000]=1
problem

array([0., 1., 0., ..., 1., 1., 0.])

In [21]:
train_x=error
train_y=problem

In [22]:
del error, problem

In [23]:
print(train_x)
print(train_y)

[[  0.   0.   8. ...   0.   0.   0.]
 [  0.   0.   0. ... 113.  56.   1.]
 [  0.   0.   2. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ...  58.   8.   5.]
 [  0.   0.   0. ...   6.   0.   0.]
 [  0.   0.   4. ...   0.   0.   0.]]
[0. 1. 0. ... 1. 1. 0.]


In [48]:
# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      {
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 50,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 10
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

[LightGBM] [Info] Number of positive: 3633, number of negative: 8367
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006628 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4353
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.302750 -> initscore=-0.834237
[LightGBM] [Info] Start training from score -0.834237
[10]	valid_0's auc: 0.791239	valid_0's pr_auc: 0.788821
[20]	valid_0's auc: 0.797482	valid_0's pr_auc: 0.79351
[30]	valid_0's auc: 0.801818	valid_0's pr_auc: 0.798739
[40]	valid_0's auc: 0.801538	valid_0's pr_auc: 0.799164
[50]	valid_0's auc: 0.80098	valid_0's pr_auc: 0.799982
[LightGBM] [Info] Number of positive: 4828, number of negative: 7172
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002

In [49]:
print(np.mean(auc_scores))

0.8027883727348663


In [50]:
test_err = pd.read_csv('test_err_data.csv')
test_err

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,20201101030227,model_1,04.16.3553,31,1
1,30000,20201101030227,model_1,04.16.3553,33,2
2,30000,20201101030228,model_1,04.16.3553,15,1
3,30000,20201101030256,model_1,04.16.3553,22,1
4,30000,20201101030300,model_1,04.16.3553,11,1
...,...,...,...,...,...,...
16532643,44998,20201130210050,model_1,04.16.3553,40,0
16532644,44998,20201130211831,model_1,04.16.3553,31,1
16532645,44998,20201130211832,model_1,04.16.3553,15,1
16532646,44998,20201130212259,model_1,04.16.3553,16,1


In [51]:
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

In [52]:
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x[person_idx - test_user_id_min,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
print(test_x.shape)

100%|██████████████████████████████████████████████████████████████████| 16532648/16532648 [01:40<00:00, 164660.95it/s]

(14999, 42)


In [53]:
pred_y_list = []
for model in models:
    pred_y = model.predict(test_x)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [54]:
pred_ensemble

array([[0.90949959],
       [0.201073  ],
       [0.273998  ],
       ...,
       [0.48851148],
       [0.84239338],
       [0.35692521]])

In [55]:
sample_submssion = pd.read_csv('sample_submission.csv')
sample_submssion['problem'] = pred_ensemble.reshape(-1)
sample_submssion.to_csv("dacon_baseline.csv", index = False)
sample_submssion

,user_id,problem
0,30000,0.909500
1,30001,0.201073
2,30002,0.273998
3,30003,0.733823
4,30004,0.614840
...,...,...
14994,44994,0.290697
14995,44995,0.312142
14996,44996,0.488511
14997,44997,0.842393
